In [26]:
import os
import zipfile
import requests

import pandas as pd
import numpy as np
import fiona
import geopandas as gpd

from sqlalchemy import create_engine

In [2]:
# engine.execute('create extension postgis;')

In [2]:
response = requests.get('https://api.os.uk/downloads/v1/products')
products = pd.DataFrame(response.json())
products['download'] = products.url.apply(lambda x: requests.get(x).json()['downloadsUrl'])
products['download2'] = products.download.apply(lambda x: requests.get(x).json())
products['formats'] = products.download2.apply(lambda x: [e['format'] for e in x])
products['geopackage'] = products.formats.apply(lambda x: 'GeoPackage' in x)
products['geopackage_url'] = products.apply(lambda x: [e['url'] for e in x.download2 
                                                       if e['format'] == 'GeoPackage'][0] 
                                            if x.geopackage else None, axis=1)

In [27]:
def download_row(row, _format):
    print(row.id)
    try:
        [url] = [e['url'] for e in row.download2 if e['format'].split(' ')[-1] == _format]
        with open(f'../data/{row.id}{_format}.zip', 'wb') as downloading:
            downloading.write(requests.get(url).content)
            downloading.close()
        return "Done!"
    except Exception as e:
        print(e)

In [56]:
products['downloaded'] = products.apply(
    lambda x: download_row(x, 'GeoPackage') if x.geopackage\
    else download_row(x, 'Shapefile'), 
    axis=1
)
    

In [3]:
def download_geopackage(row):
    print(row.id)
    with open(f'../data/{row.id}_gpkg.zip', 'wb') as downloading:
        downloading.write(requests.get(row.geopackage_url).content)
        downloading.close()
    return "Done!"

In [7]:
products[['id','description','formats']]

,id,description,formats
0,250kScaleColourRaster,"Get the regional view of towns and villages, r...",[TIFF-LZW]
1,BoundaryLine,"From Euro constituencies to council wards, Bou...","[ESRI® Shapefile, GML, GeoPackage, MapInfo® TA..."
2,CodePointOpen,Free and open postcode location data. Can be u...,"[CSV, GeoPackage]"
3,GBOverviewMaps,Our simplest maps of the British Isles.,[GeoTIFF]
4,LIDS,A comprehensive dataset of cross-referenced id...,"[CSV, CSV, CSV, CSV, CSV, CSV, CSV, CSV, CSV, ..."
5,MiniScale,A simple overview map of Great Britain.,"[Zip file (containing EPS, Illustrator and TIF..."
6,OpenGreenspace,Covering a range of greenspaces in urban and r...,"[ESRI® Shapefile, ESRI® Shapefile, ESRI® Shape..."
7,OpenMapLocal,"Map, visualise and truly understand your data ...","[ESRI® Shapefile, ESRI® Shapefile, ESRI® Shape..."
8,OpenNames,"A comprehensive dataset of place names, roads ...","[CSV, GML, GeoPackage]"
9,OpenRivers,Understand how watercourses in Great Britain j...,"[ESRI® Shapefile, GML, GeoPackage, Vector Tiles]"


In [30]:
products[14:15].apply(lambda x: download_row(x, 'GeoPackage'), axis=1)

OpenZoomstack


14    Done!
dtype: object

In [8]:
products.description.values

array(['Get the regional view of towns and villages, roads and places of interest.',
       'From Euro constituencies to council wards, Boundary-Line™ maps every administrative boundary in detail for you.',
       'Free and open postcode location data. Can be used for geographical analysis, simple route planning, asset management and much more.',
       'Our simplest maps of the British Isles.',
       'A comprehensive dataset of cross-referenced identifiers, between various OS data products.',
       'A simple overview map of Great Britain.',
       'Covering a range of greenspaces in urban and rural areas including playing fields, sports’ facilities, play areas and allotments.',
       'Map, visualise and truly understand your data at street level.',
       'A comprehensive dataset of place names, roads numbers and postcodes for Great Britain.',
       'Understand how watercourses in Great Britain join up.',
       'Get a high-level view of the road network, from motorways to country

In [21]:
{e for x in list(products.formats) for e in x}

{'ASCII Grid and GML (Grid)',
 'CSV',
 'DXF',
 'ESRI® Shapefile',
 'GML',
 'GeoPackage',
 'GeoTIFF',
 'MapInfo® TAB',
 'TIFF-LZW',
 'Vector Tiles',
 'Zip file (containing EPS, Illustrator and TIFF-LZW)'}